In [1]:
%pip install wandb
%pip install graphviz
%pip install torchviz
import wandb
wandb.login()#doesnt detect WANDB_NOTEBOOK_NAME on windows

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tetetoutou (simclr-doctoral-research). Use `wandb login --relogin` to force relogin


True

In [2]:
wandb.init(project="test-project", entity="simclr-doctoral-research")

In [3]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}

In [10]:
import os
import wandb
import torch
import torchvision
import torch.nn.functional as F
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = F.sigmoid(self.t_conv2(x))
        return x





In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = autoencoder().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        #img = img.view(img.size(0), -1)//makes the image 3D i think
        img = Variable(img).to(device)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.item()))
    #if epoch % 10 == 0:
        #pic = to_img(output.cpu().data)
        #save_image(pic, './mlp_img/image_{}.png'.format(epoch))
    wandb.log({"loss": loss})

    wandb.watch(model)
    # print('epoch [{}/{}], loss:{:.4f}'
    # #       .format(epoch + 1, num_epochs, loss.data[0]))
    # if epoch % 10 == 0:
    #     pic = to_img(output.cpu().data)
    #     save_image(pic, './mlp_img/image_{}.png'.format(epoch))




epoch [1/100], loss:0.9249
epoch [2/100], loss:0.9080
epoch [3/100], loss:0.8764
epoch [4/100], loss:0.8688
epoch [5/100], loss:0.8692
epoch [6/100], loss:0.8647
epoch [7/100], loss:0.8669
epoch [8/100], loss:0.8589
epoch [9/100], loss:0.8537
epoch [10/100], loss:0.8506
epoch [11/100], loss:0.8512
epoch [12/100], loss:0.8630
epoch [13/100], loss:0.8604
epoch [14/100], loss:0.8548
epoch [15/100], loss:0.8526
epoch [16/100], loss:0.8560
epoch [17/100], loss:0.8553
epoch [18/100], loss:0.8609
epoch [19/100], loss:0.8504
epoch [20/100], loss:0.8485
epoch [21/100], loss:0.8497
epoch [22/100], loss:0.8504
epoch [23/100], loss:0.8408
epoch [24/100], loss:0.8516
epoch [25/100], loss:0.8501
epoch [26/100], loss:0.8545
epoch [27/100], loss:0.8460
epoch [28/100], loss:0.8506
epoch [29/100], loss:0.8473
epoch [30/100], loss:0.8474
epoch [31/100], loss:0.8522
epoch [32/100], loss:0.8490
epoch [33/100], loss:0.8517
epoch [34/100], loss:0.8526
epoch [35/100], loss:0.8660
epoch [36/100], loss:0.8507
e

In [ ]:
from torchviz import make_dot
y = model(img)
make_dot(y, params=dict(list(model.named_parameters()))).render("torchviz", format="png")

'torchviz.png'